Install necessary libraries

In [ ]:
!pip install scikeras

importing libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,  roc_auc_score
from sklearn.impute import SimpleImputer
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam, Adadelta, RMSprop
from keras.utils import to_categorical
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV,cross_val_predict, KFold
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
import pickle as pk

Mounted my google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/CustomerChurn_dataset.csv')

displaying the dataframe

In [ ]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


Display first five rows of DataFrame

In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Display the data types of the columns

In [ ]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Convert 'Total charges' data type to numeric  and used "coerce" with handles errors by setting them to NaN.

In [ ]:
df['TotalCharges']=pd.to_numeric(df['TotalCharges'], errors="coerce")

Display data types of each column to ensure conversion worked.

In [ ]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

Checked for missing values

In [ ]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

Displaying information about the dataframe

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Display descriptive statistics

In [ ]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


Filled the missing values in 'Total Charges' column using backward fill.

In [ ]:
df['TotalCharges'] = df['TotalCharges'].fillna(method='bfill')

Display updated information after filling the missing values.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Checked for missing values

In [ ]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Converting categorical columns to numerical columns using factorization

In [ ]:
categorical_columns= df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df[column], _ = pd.factorize(df[column])

Display dataframe to echeck if factorization worked

In [ ]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,29.85,29.85,0
1,1,1,0,1,0,34,1,1,0,1,...,1,0,0,0,1,1,1,56.95,1889.50,0
2,2,1,0,1,0,2,1,1,0,1,...,0,0,0,0,0,0,1,53.85,108.15,1
3,3,1,0,1,0,45,0,0,0,1,...,1,1,0,0,1,1,2,42.30,1840.75,0
4,4,0,0,1,0,2,1,1,1,0,...,0,0,0,0,0,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,7038,1,0,0,1,24,1,2,0,1,...,1,1,1,1,1,0,1,84.80,1990.50,0
7039,7039,0,0,0,1,72,1,2,1,0,...,1,0,1,1,1,0,3,103.20,7362.90,0
7040,7040,0,0,0,1,11,0,0,0,1,...,0,0,0,0,0,0,0,29.60,346.45,0
7041,7041,1,1,0,0,4,1,2,1,0,...,0,0,0,0,0,0,1,74.40,306.60,1


Separating features(X) and target variiable(y)

In [ ]:
x= df.drop('Churn', axis=1)
x=x.drop('customerID', axis=1)
y= df['Churn']

Initialize standard scaler

In [ ]:
scaler= StandardScaler()

fit the scaler on the data and transform

In [ ]:
X_scaled =pd.DataFrame(scaler.fit_transform(x), columns= x.columns)

concatenate scaled features and target variable

In [ ]:
scaled_df= pd.concat([X_scaled, y],axis=1)

display datafram with scaled features

In [ ]:
scaled_df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,-1.009559,-0.439916,-1.034530,-0.654012,-1.277445,-3.054010,-2.061669,-1.183234,-0.903589,-1.180777,-0.998016,-0.909172,-1.071457,-1.078210,-0.828207,-0.829798,-1.145198,-1.160323,-0.993556,0
1,0.990532,-0.439916,0.966622,-0.654012,0.066327,0.327438,-0.505697,-1.183234,0.351386,0.173655,0.286059,-0.909172,-1.071457,-1.078210,0.371271,1.205113,-0.274744,-0.259629,-0.173106,0
2,0.990532,-0.439916,0.966622,-0.654012,-1.236724,0.327438,-0.505697,-1.183234,0.351386,-1.180777,-0.998016,-0.909172,-1.071457,-1.078210,-0.828207,-0.829798,-0.274744,-0.362660,-0.959011,1
3,0.990532,-0.439916,0.966622,-0.654012,0.514251,-3.054010,-2.061669,-1.183234,0.351386,0.173655,0.286059,0.347362,-1.071457,-1.078210,0.371271,1.205113,0.595711,-0.746535,-0.194613,0
4,-1.009559,-0.439916,0.966622,-0.654012,-1.236724,0.327438,-0.505697,0.172250,-0.903589,0.173655,-0.998016,-0.909172,-1.071457,-1.078210,-0.828207,-0.829798,-1.145198,0.197365,-0.939819,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.990532,-0.439916,-1.034530,1.529024,-0.340876,0.327438,1.050276,-1.183234,0.351386,0.173655,0.286059,0.347362,0.238887,0.234693,0.371271,-0.829798,-0.274744,0.665992,-0.128546,0
7039,-1.009559,-0.439916,-1.034530,1.529024,1.613701,0.327438,1.050276,0.172250,-0.903589,-1.180777,0.286059,-0.909172,0.238887,0.234693,0.371271,-0.829798,1.466165,1.277533,2.241677,0
7040,-1.009559,-0.439916,-1.034530,1.529024,-0.870241,-3.054010,-2.061669,-1.183234,0.351386,0.173655,-0.998016,-0.909172,-1.071457,-1.078210,-0.828207,-0.829798,-1.145198,-1.168632,-0.853876,0
7041,0.990532,2.273159,-1.034530,-0.654012,-1.155283,0.327438,1.050276,0.172250,-0.903589,0.173655,-0.998016,-0.909172,-1.071457,-1.078210,-0.828207,-0.829798,-0.274744,0.320338,-0.871458,1


In [ ]:
scaled_df.describe()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7.043000e+03,7043.000000
mean,-2.219500e-17,-4.842546e-17,-6.557614e-18,1.069396e-16,-2.421273e-17,7.869137e-17,1.402321e-16,8.575342e-18,7.440370e-17,5.498307e-17,6.708944e-17,-9.180660e-17,-1.109750e-17,-6.254955e-17,-7.566478e-17,-3.026591e-17,-1.916841e-17,-6.406285e-17,-1.916841e-17,0.265370
std,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,0.441561
min,-1.009559e+00,-4.399165e-01,-1.034530e+00,-6.540119e-01,-1.318165e+00,-3.054010e+00,-2.061669e+00,-1.183234e+00,-9.035893e-01,-1.180777e+00,-9.980158e-01,-9.091720e-01,-1.071457e+00,-1.078210e+00,-8.282069e-01,-8.297975e-01,-1.145198e+00,-1.545860e+00,-9.984306e-01,0.000000
25%,-1.009559e+00,-4.399165e-01,-1.034530e+00,-6.540119e-01,-9.516817e-01,3.274383e-01,-5.056967e-01,-1.183234e+00,-9.035893e-01,-1.180777e+00,-9.980158e-01,-9.091720e-01,-1.071457e+00,-1.078210e+00,-8.282069e-01,-8.297975e-01,-1.145198e+00,-9.725399e-01,-8.303391e-01,0.000000
50%,9.905318e-01,-4.399165e-01,9.666223e-01,-6.540119e-01,-1.372744e-01,3.274383e-01,-5.056967e-01,1.722502e-01,3.513860e-01,1.736550e-01,2.860590e-01,3.473622e-01,2.388872e-01,2.346933e-01,-8.282069e-01,-8.297975e-01,-2.747437e-01,1.857327e-01,-3.907201e-01,0.000000
75%,9.905318e-01,-4.399165e-01,9.666223e-01,1.529024e+00,9.214551e-01,3.274383e-01,1.050276e+00,1.722502e-01,3.513860e-01,1.736550e-01,2.860590e-01,3.473622e-01,2.388872e-01,2.346933e-01,3.712710e-01,1.205113e+00,5.957106e-01,8.338335e-01,6.667124e-01,1.000000
max,9.905318e-01,2.273159e+00,9.666223e-01,1.529024e+00,1.613701e+00,3.274383e-01,1.050276e+00,1.527734e+00,1.606361e+00,1.528087e+00,1.570134e+00,1.603896e+00,1.549232e+00,1.547597e+00,1.570749e+00,1.205113e+00,1.466165e+00,1.794352e+00,2.824880e+00,1.000000


FEATURE IMPORTANCE

Separate features (X) and target variable (y) from scaled DataFrame

In [ ]:
X= scaled_df.drop('Churn', axis=1)
y= scaled_df['Churn']

Split the data into training and testing sets

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

 Train a Random Forest Classifier

In [ ]:
randomF=RandomForestClassifier(n_estimators=48,n_jobs=-1)

In [ ]:
randomF.fit(Xtrain, Ytrain)

RandomForestClassifier(n_estimators=48, n_jobs=-1)

Make predictions on the test set

In [ ]:
y_pred=randomF.predict(Xtest)

Calculate and print accuracy score

In [ ]:
accuracy_score(y_pred,Ytest)

0.7849538679914834

Display feature importance

In [ ]:
for name, score in zip(scaled_df.columns[:-1], randomF.feature_importances_):
  print(name, score)

gender 0.02739131543594721
SeniorCitizen 0.02021318177307455
Partner 0.021874328378090977
Dependents 0.01917172855483
tenure 0.15438139172493165
PhoneService 0.004740732120363866
MultipleLines 0.02432328532768123
InternetService 0.02349121807916491
OnlineSecurity 0.04812312819015977
OnlineBackup 0.026612637108600068
DeviceProtection 0.021804163440930537
TechSupport 0.03577299377496127
StreamingTV 0.022779643007785143
StreamingMovies 0.01823048380703618
Contract 0.09254400062726073
PaperlessBilling 0.025309043641151347
PaymentMethod 0.055428611107763155
MonthlyCharges 0.17354988694678666
TotalCharges 0.18425822695348085


In [ ]:
feature_importance = randomF.feature_importances_

In [ ]:
feature_names=Xtrain.columns

In [ ]:
feature_importance_dict = dict (zip (feature_names, feature_importance))

Sort features by importance in descending order

In [ ]:
sorted_F = sorted(feature_importance_dict. items (), key=lambda x: x[1], reverse=True)

Select the top 12 features


In [ ]:
selected_F = sorted_F[: 12]

In [ ]:
selected_Fnames = [feature[0] for feature in selected_F]

In [ ]:
selected_Fnames

['TotalCharges',
 'MonthlyCharges',
 'tenure',
 'Contract',
 'PaymentMethod',
 'OnlineSecurity',
 'TechSupport',
 'gender',
 'OnlineBackup',
 'PaperlessBilling',
 'MultipleLines',
 'InternetService']

In [ ]:
selected_Fnames

['TotalCharges',
 'MonthlyCharges',
 'tenure',
 'Contract',
 'PaymentMethod',
 'OnlineSecurity',
 'TechSupport',
 'gender',
 'OnlineBackup',
 'PaperlessBilling',
 'MultipleLines',
 'InternetService']

Multilayer Perceptron (MLP) - Grid Search and KFold

In [ ]:
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

Function to create a model for grid search

In [ ]:
def create_model(optimizer='adam'):
    input_layer = Input(shape=(Xtrain.shape[1],))
    hidden_layer_1 = Dense(32, activation='relu')(input_layer)
    hidden_layer_2 = Dense(24, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(12, activation='relu')(hidden_layer_2)
    output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

Wrapped the model for scikit-learn

In [ ]:
model = KerasClassifier(model=create_model)

Define the parameter grid for grid search

In [ ]:
param_grid = {
    'optimizer': ['adam','adadelta'],
    'epochs': [10,20,50]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=kfold)


Perform grid search

In [ ]:
grid_r = grid.fit(Xtrain, Ytrain,batch_size=32, validation_data=(Xtest, Ytest))

Epoch 1/10
118/118 [==============================] - 1s 3ms/step - loss: 0.5896 - accuracy: 0.6709 - val_loss: 0.4710 - val_accuracy: 0.7693
Epoch 2/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4470 - accuracy: 0.7857 - val_loss: 0.4379 - val_accuracy: 0.7857
Epoch 3/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4287 - accuracy: 0.7958 - val_loss: 0.4353 - val_accuracy: 0.7942
Epoch 4/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4233 - accuracy: 0.7979 - val_loss: 0.4284 - val_accuracy: 0.7921
Epoch 5/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4191 - accuracy: 0.8009 - val_loss: 0.4261 - val_accuracy: 0.7892
Epoch 6/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4150 - accuracy: 0.8067 - val_loss: 0.4290 - val_accuracy: 0.7892
Epoch 7/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8091 - val_loss: 0.4271 - val_accuracy: 0.7963
Epoch 

Print best parameters and accuracy score

In [ ]:
print(f"Best parameters: {grid_r.best_params_}")
print(f"Best accuracy score: {grid_r.best_score_}")

Best parameters: {'epochs': 10, 'optimizer': 'adam'}
Best accuracy score: 0.8001419950301739


In [ ]:
best_parameters=grid_r.best_params_
input_layer = Input(shape=(Xtrain.shape[1],))
hidden_layer_1 = Dense(32, activation='relu')(input_layer)
hidden_layer_2 = Dense(24, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(12, activation='relu')(hidden_layer_2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

best_model = Model(inputs=input_layer, outputs=output_layer)
best_model.compile(optimizer=grid_r.best_params_['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])
best_model.fit(Xtrain, Ytrain,epochs=grid_r.best_params_['epochs'], validation_data=(Xtest, Ytest))


Epoch 1/10
177/177 [==============================] - 1s 2ms/step - loss: 0.5430 - accuracy: 0.7036 - val_loss: 0.4466 - val_accuracy: 0.7736
Epoch 2/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4266 - accuracy: 0.7975 - val_loss: 0.4352 - val_accuracy: 0.7857
Epoch 3/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4180 - accuracy: 0.8009 - val_loss: 0.4325 - val_accuracy: 0.7935
Epoch 4/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4143 - accuracy: 0.8035 - val_loss: 0.4339 - val_accuracy: 0.7885
Epoch 5/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4107 - accuracy: 0.8062 - val_loss: 0.4313 - val_accuracy: 0.7892
Epoch 6/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4088 - accuracy: 0.8092 - val_loss: 0.4346 - val_accuracy: 0.7871
Epoch 7/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4082 - accuracy: 0.8056 - val_loss: 0.4308 - val_accuracy: 0.7871
Epoch 

Get the best model from grid search and Train the best model on the entire training set

In [ ]:
best_model.fit(Xtrain, Ytrain, epochs=grid_r.best_params_['epochs'], batch_size=32, verbose=0)

In [ ]:
best_model

KerasClassifier(
	model=<function create_model at 0x7a92962611b0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	class_weight=None
)

Make predictions on the test set using MLP. Calculated and printed the AUC Score (Area Under the ROC Curve)

In [ ]:
# Make predictions on the test set using MLP
Y_pred_proba_mlp = best_model.predict(Xtest)
Y_pred_binary_mlp = (Y_pred_proba_mlp > 0.5).astype(int)

# Calculate and print the AUC Score (Area Under the ROC Curve)
auc_score_mlp = roc_auc_score(Ytest, Y_pred_proba_mlp)
print(f"AUC Score (MLP): {auc_score_mlp}")


45/45 [==============================] - 0s 2ms/step
AUC Score (MLP): 0.8335064196956781


Function to create model for grid search

In [ ]:
def create_model1(optimizer='adam',dropout_rate=0.2):
    input_layer = Input(shape=(Xtrain.shape[1],))
    hidden_layer_1 = Dense(32, activation='relu')(input_layer)
    hidden_layer_2 = Dense(24, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(12, activation='relu')(hidden_layer_2)
    output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)


    model = Model(inputs=input_layer, outputs=output_layer)
    # compile the model with specified optimizer, loss function snd evaluation metric
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

wrap the model in scikit-learn

In [ ]:
model = KerasClassifier(model=create_model1,dropout_rate=0.2)

In [ ]:
# defining parameters for grid
param_grid = {
    'optimizer': ['adam'],
    'epochs': [10],
    'dropout_rate':[0.2,0.3],

}

grid_s = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=kfold)

In [ ]:
grid_result = grid_s.fit(Xtrain, Ytrain,batch_size=32, validation_data=(Xtest, Ytest))

Epoch 1/10
118/118 [==============================] - 1s 3ms/step - loss: 0.4976 - accuracy: 0.7718 - val_loss: 0.4445 - val_accuracy: 0.7835
Epoch 2/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4336 - accuracy: 0.7966 - val_loss: 0.4329 - val_accuracy: 0.7942
Epoch 3/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4247 - accuracy: 0.8017 - val_loss: 0.4306 - val_accuracy: 0.7892
Epoch 4/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8030 - val_loss: 0.4291 - val_accuracy: 0.7913
Epoch 5/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4173 - accuracy: 0.8024 - val_loss: 0.4251 - val_accuracy: 0.7928
Epoch 6/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8110 - val_loss: 0.4257 - val_accuracy: 0.7921
Epoch 7/10
118/118 [==============================] - 0s 2ms/step - loss: 0.4122 - accuracy: 0.8078 - val_loss: 0.4256 - val_accuracy: 0.7935
Epoch 

Display the best accuracy score after optimization using the best model from the first grid search.

In [ ]:
print(f"Best parameters: {grid_result.best_params_}")
print(f"Best accuracy score: {grid_result.best_score_}")

Best parameters: {'dropout_rate': 0.2, 'epochs': 10, 'optimizer': 'adam'}
Best accuracy score: 0.8001419950301739


Make predictions using mlp and finding auc score for the optimized model.

In [ ]:
parameters=grid_result.best_params_

In [ ]:
input_layer = Input(shape=(Xtrain.shape[1],))
hidden_layer_1 = Dense(32, activation='relu')(input_layer)
hidden_layer_2 = Dense(24, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(12, activation='relu')(hidden_layer_2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

optimized_best_model = Model(inputs=input_layer, outputs=output_layer)
optimized_best_model.compile(optimizer=parameters['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])
optimized_best_model.fit(Xtrain, Ytrain,epochs=parameters['epochs'], validation_data=(Xtest, Ytest))

Epoch 1/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4656 - accuracy: 0.7662 - val_loss: 0.4434 - val_accuracy: 0.7850
Epoch 2/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4264 - accuracy: 0.8001 - val_loss: 0.4313 - val_accuracy: 0.7921
Epoch 3/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4172 - accuracy: 0.8060 - val_loss: 0.4318 - val_accuracy: 0.7899
Epoch 4/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4142 - accuracy: 0.8088 - val_loss: 0.4243 - val_accuracy: 0.7991
Epoch 5/10
177/177 [==============================] - 2s 10ms/step - loss: 0.4101 - accuracy: 0.8106 - val_loss: 0.4301 - val_accuracy: 0.7921
Epoch 6/10
177/177 [==============================] - 1s 6ms/step - loss: 0.4087 - accuracy: 0.8115 - val_loss: 0.4260 - val_accuracy: 0.7956
Epoch 7/10
177/177 [==============================] - 1s 6ms/step - loss: 0.4061 - accuracy: 0.8149 - val_loss: 0.4237 - val_accuracy: 0.7949
Epoch

In [ ]:
# Make predictions on the test set using MLP
Y_pred_proba_mlp = optimized_best_model.predict(Xtest)
Y_pred_binary_mlp = (Y_pred_proba_mlp > 0.5).astype(int)

# Calculate and print the AUC Score (Area Under the ROC Curve)
auc_score_mlp = roc_auc_score(Ytest, Y_pred_proba_mlp)
print(f"AUC Score (MLP): {auc_score_mlp}")


45/45 [==============================] - 0s 942us/step
AUC Score (MLP): 0.8361879666227493


In [ ]:
# saving trained model
file_name = "/content/drive/My Drive/Colab Notebooks/optimized_best_model.pkl"
with open(file_name, "wb") as file:
  pk.dump(optimized_best_model,file)


with open('/content/drive/MyDrive/scaler_model2.pkl', 'wb') as scaler_file:
    pk.dump(scaler, scaler_file)

loaded_model = pk.load(open(file_name, 'rb'))
